In [1]:
import datetime as dt

In [15]:
FECHA_INICIO = dt.date(2018, 3, 30)
FECHA_FIN = dt.date(2018, 6, 27)
UN_DIA = dt.timedelta(days=1)
SITIO_BASE = 'https://lopezobrador.org.mx/'
print('Inicio de campaña: ', FECHA_INICIO)
print('Fin de campaña: ', FECHA_FIN)

Inicio de campaña:  2018-03-30
Fin de campaña:  2018-06-27


In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
driver = webdriver.Chrome("C:\\Chromedriver\\chromedriver.exe")

In [5]:
artículos = []


In [6]:
def obtener_ruta(fecha):
    return SITIO_BASE + fecha.strftime("%Y/%m/%d")

In [47]:
fecha_actual = FECHA_INICIO
fecha_actual += UN_DIA*2
while fecha_actual <= FECHA_FIN:
    ruta_actual = obtener_ruta(fecha_actual)
    driver.get(ruta_actual)
    content = driver.page_source
    soup = BeautifulSoup(content)
    # Buscamos todos los artículos con clase boletin
    for soup_articulo in soup.findAll('article', attrs={'class':'category-boletines'}):
        articulo = {
            'fuente' : '',
            'encabezado' : '',
            'sumarios' : [],
            'contenido' : [],
        }
        soup_encabezado = soup_articulo.find('h2', attrs={'class':'entry-title'})
        articulo['encabezado'] = soup_encabezado.a.string
        articulo['fuente'] = soup_encabezado.a.attrs['href']
        soup_content = soup_articulo.find('div', attrs={'class':'entry-content'})
        for soup_p in soup_content.findAll('p'):
            contenido = soup_p.string if soup_p.string else ""
            if( len(contenido) < 3 ):
                contenido = " ".join([child.string for child in soup_p.children if child.string])
            if( len(contenido) >= 3 ):
                articulo['contenido'].append(contenido)
    fecha_actual += UN_DIA
    break

None
None
Descargar audio  discurso AMLO
None
 
None
Amigas y amigos:
Iniciamos la campaña, aquí donde comienza nuestra patria, en Ciudad Juárez. iniciamos aquí la campaña como homenaje a este legendario Paso del Norte, donde se refugió y resistió Juárez y su gabinete, en los tiempos de la lucha contra los conservadores y la Invasión Francesa.
Medio siglo después, en mayo de 1911, aquí también tuvo lugar la batalla decisiva, en la cual los revolucionarios, encabezados por Francisco I. Madero, con el apoyo militar de Pascual Orozco y Francisco Villa, derrotaron al ejército federal para propiciar la caída del régimen dictatorial porfirista.
De modo que Ciudad Juárez, antes Paso del Norte, ha sido testigo de hechos históricos trascendentes y punto de partida de importantes transformaciones. Por eso, decidimos iniciar nuestra campaña aquí en este sitio, por demás emblemático, aquí al pie de la estatua del mejor presidente que ha habido en la historia de nuestro país, Benito Juárez García.


In [48]:
articulo

{'fuente': 'https://lopezobrador.org.mx/2018/04/01/discurso-de-amlo-al-iniciar-su-campana-en-ciudad-juarez-chihuahua/',
 'encabezado': 'Discurso AMLO al iniciar campaña en Ciudad Juárez, Chihuahua',
 'sumarios': [],
 'contenido': ['\xa0  Boletín campaña-001',
  'Descargar audio\xa0 discurso AMLO',
  'Ciudad Juárez, Chihuahua, 01 de abril de 2018.-  \xa0 Palabras de Andrés Manuel López Obrador, candidato a la Presidencia de la República por la Coalición: ”Juntos haremos historia”, en su inicio de campaña en Ciudad Juárez, Chihuahua.',
  'Amigas y amigos:',
  'Iniciamos la campaña, aquí donde comienza nuestra patria, en Ciudad Juárez. iniciamos aquí la campaña como homenaje a este legendario Paso del Norte, donde se refugió y resistió Juárez y su gabinete, en los tiempos de la lucha contra los conservadores y la Invasión Francesa.',
  'Medio siglo después, en mayo de 1911, aquí también tuvo lugar la batalla decisiva, en la cual los revolucionarios, encabezados por Francisco I. Madero, co

In [35]:
" ".join(['as', 'djsak', 's'])

'as djsak s'